In [24]:
import numpy as np
import pandas as pd
import re
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans
url = "https://github.com/swarj/MLAssignment3/blob/main/usnewshealth.xlsx?raw=true"
raw_df = pd.read_excel(url)
raw_df.columns = ['Tweet Content']
#s1 = pd.Series([])
my_list = []
#df.style
newdf = raw_df['Tweet Content'].str.split('|').str[2]
#df['Tweet Content'] = df['Tweet Content'].re.sub('|').str[2]
#newdf
k_values = [5, 10, 15, 20, 30, 50]
newdf.columns = ['Tweets']
for indx in newdf:
  indx = re.sub('@[\w]+','', indx)
  indx = re.sub("#","", indx)
  indx = re.sub(r'http\S+', '', indx)
  indx = indx.lower()
  my_list.append(indx)

# pre-processed dataframe
df = pd.DataFrame(my_list)

s = len(df)
results = []
#TF-IDF vectors for tweets
vectorizer = TfidfVectorizer()
#set the vectors into an np array
t_vectors = vectorizer.fit_transform(df[0]).toarray()
#jaccard distance values
ja = np.zeros((s, s))
#iterate on the size of the dataframe
for i in range(s):
    for j in range(i, s):
        ja[i][j] = (1 - np.sum(np.minimum(t_vectors[i], t_vectors[j])) / np.sum(np.maximum(t_vectors[i], t_vectors[j])))
        ja[j][i] = ja[i][j]

#repeat for all the k requested
for k in k_values:
  #select centroid indices to begin working 
  c_index = np.random.choice(s, k, replace=False)   
  #assign the vectors to the randomly selected tweets
  ce = t_vectors[c_index]
  temp = np.zeros(s)
  #number of iterations. This value was changed for testing purposes
  for m in range(5):
      #iterate over the tfid vectors to calculate each distance with the centroid
      for i, t in enumerate(t_vectors):
          #min distance between the selected tween and the centroid
          min_distance = [1 - np.sum(np.minimum(t, c)) / np.sum(np.maximum(t, c)) for c in ce]
          #assign the cluster for that distance
          temp[i] = np.argmin(min_distance)
      #for all centroids
      for j in range(k):
          #get the tweets for each of the clusters
          tweets = t_vectors[temp == j]
          #update the centroid
          ce[j] = np.mean(tweets, axis=0)
  #get the size of all the clusters
  sizes = np.bincount(temp.astype(int))

  #iterate for each of the clusters and print the size for each
  for i in range(k):
    print(f"For K value {k} - Cluster {i+1} has {sizes[i]} tweets") 
  #calculate SSE
  error = np.zeros(k)
  #iterate for each of the clusters
  for j in range(k):
    #select the tweets for the given cluster
    tweets = t_vectors[temp == j]
    #calculate the error for each of the tweets
    error[j] = np.sum(np.linalg.norm(tweets - ce[j], axis=1) ** 2)
  #sum up the values of error to get a total for each cluster
  error = np.sum(error)
  print(f"\nThe value for SSE is: {error}\n")

For K value 5 - Cluster 1 has 385 tweets
For K value 5 - Cluster 2 has 275 tweets
For K value 5 - Cluster 3 has 141 tweets
For K value 5 - Cluster 4 has 402 tweets
For K value 5 - Cluster 5 has 155 tweets

The value for SSE is: 1305.9717054198284

For K value 10 - Cluster 1 has 94 tweets
For K value 10 - Cluster 2 has 112 tweets
For K value 10 - Cluster 3 has 171 tweets
For K value 10 - Cluster 4 has 115 tweets
For K value 10 - Cluster 5 has 185 tweets
For K value 10 - Cluster 6 has 147 tweets
For K value 10 - Cluster 7 has 123 tweets
For K value 10 - Cluster 8 has 100 tweets
For K value 10 - Cluster 9 has 206 tweets
For K value 10 - Cluster 10 has 105 tweets

The value for SSE is: 1286.9729282821675

For K value 15 - Cluster 1 has 155 tweets
For K value 15 - Cluster 2 has 122 tweets
For K value 15 - Cluster 3 has 74 tweets
For K value 15 - Cluster 4 has 58 tweets
For K value 15 - Cluster 5 has 61 tweets
For K value 15 - Cluster 6 has 99 tweets
For K value 15 - Cluster 7 has 67 tweets
